<a href="https://colab.research.google.com/github/Sam-Kon/hello-world/blob/master/Models_AudioFlux_BFCCs_100_Components_(UMAP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-up

In [ ]:
# connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# imports
import numpy as np
import pandas as pd
import librosa as lb
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def getListIndex(nrow, ncol,row_pos, col_pos):
    list_pos = row_pos*ncol + col_pos
    return(list_pos)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# Import BFCC data + tonic annotations

In [ ]:
worksheet = gc.open('AudioFlux BFCC Data').sheet1

# get_all_values gives a list of rows.
dataset = worksheet.get_all_values()

dataset = pd.DataFrame.from_records(dataset)

In [ ]:
label = dataset[dataset. columns[0]]
label.value_counts()

C#3 Scale (1.5 Kattai)    247
D3 Scale (2 Kattai)       236
D#3 Scale (2.5 Kattai)    189
C3 Scale (1 Kattai)       183
E3 Scale (3 Kattai)       124
F3 Scale (4 Kattai)        91
G#3 Scale (5.5 Kattai)     90
G3 Scale (5 Kattai)        89
A#3 Scale (6.5 Kattai)     81
F#3 Scale (4.5 Kattai)     62
B2 Scale (7 Kattai)        59
A3 Scale (6 Kattai)        57
A2 Scale (6 Kattai)        32
E4 Scale (3 Kattai)        21
D#4 Scale (2.5 Kattai)     15
C4 Scale (1 Kattai)        15
B3 Scale (7 Kattai)        12
C#4 Scale (1.5 Kattai)     12
D4 Scale (2 Kattai)        10
A#2 Scale (6.5 Kattai)      9
Name: 0, dtype: int64

# UMAP

In [ ]:
dataset.replace('',0,inplace=True)

In [ ]:
dataset = dataset.drop(labels = dataset.columns[0],axis = 1)

In [ ]:
dataset = dataset.transpose()
dataset.reset_index(inplace=True, drop=True)
dataset = dataset.transpose()

In [ ]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82807 sha256=9ba1303ecf73573c6cd8af736020dfecb1be9dee8fdb715a59562711680205ca
  Stored in directory: /root/.cache/pip/wheels/a0/e8/c6/a37ea663620bd5200ea1ba0907ab3c217042c1d035ef606acc
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=d40aba6ed47eb0ba24903566fe061178c6ef6e5947e6efe679dced334915aad1
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [ ]:
import umap

In [ ]:
labToNum = {"A2 Scale (6 Kattai)":0,
            "A#2 Scale (6.5 Kattai)":1,
            "B2 Scale (7 Kattai)":2,
            "C3 Scale (1 Kattai)":3,
            "C#3 Scale (1.5 Kattai)":4,
            "D3 Scale (2 Kattai)":5,
            "D#3 Scale (2.5 Kattai)":6,
            "E3 Scale (3 Kattai)":7,
            "F3 Scale (4 Kattai)":8,
            "F#3 Scale (4.5 Kattai)":9,
            "G3 Scale (5 Kattai)":10,
            "G#3 Scale (5.5 Kattai)":11,
            "A3 Scale (6 Kattai)":12,
            "A#3 Scale (6.5 Kattai)":13,
            "B3 Scale (7 Kattai)":14,
            "C4 Scale (1 Kattai)":15,
            "C#4 Scale (1.5 Kattai)":16,
            "D4 Scale (2 Kattai)":17,
            "D#4 Scale (2.5 Kattai)":18,
            "E4 Scale (3 Kattai)":19}

In [ ]:
numLabel = pd.Series()
for lab in label:
  numLabel = numLabel.append(pd.Series([labToNum.get(lab)]))

<ipython-input-13-52952770f78a>:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  numLabel = pd.Series()
<ipython-input-13-52952770f78a>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  numLabel = numLabel.append(pd.Series([labToNum.get(lab)]))
<ipython-input-13-52952770f78a>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  numLabel = numLabel.append(pd.Series([labToNum.get(lab)]))
<ipython-input-13-52952770f78a>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  numLabel = numLabel.append(pd.Series([labToNum.get(lab)]))
<ipython-input-13-52952770f78a>:3: FutureWarning: The series.append method is deprecated 

In [ ]:
numLabel.reset_index(drop=True)

0        7
1        7
2        6
3        7
4        6
        ..
1629    12
1630     3
1631     4
1632     3
1633    10
Length: 1634, dtype: int64

In [ ]:
manifold = umap.UMAP(n_components = 100, n_neighbors = 101, random_state=42).fit(dataset, y=numLabel)
data_reduced = manifold.transform(dataset)

In [ ]:
dataset = pd.DataFrame(data_reduced)
dataset.insert(0, "Tonic", label, True) # inserting the tonic as the first column

In [ ]:
count_row = dataset.shape[0]
count_col = dataset.shape[1]

# Open our sheet and add some data.
worksheet = gc.open('AF BFCC 100 UMAP Data').sheet1
#cell_list = worksheet.range('A1:M'+str(length))

# note this outputs data from the 1st row
cell_list = worksheet.range(1,1,count_row,count_col)

for row in range(0,count_row):
    for col in range(0,count_col):
        cell_list[count_col*row + col].value = str(dataset.iloc[row,col])

worksheet.update_cells(cell_list)

{'spreadsheetId': '1JzF4b9WC5o0WzdTmJQqBgIDvt8K-DiDoksJRjOl51ZM',
 'updatedRange': 'Sheet1!A1:CW1634',
 'updatedRows': 1634,
 'updatedColumns': 101,
 'updatedCells': 165034}

# Dataset Formatting

In [ ]:
worksheet = gc.open('AF BFCC 100 UMAP Data').sheet1

# get_all_values gives a list of rows.
dataset = worksheet.get_all_values()

dataset = pd.DataFrame.from_records(dataset)

label = dataset[dataset. columns[0]]
label.value_counts()

C#3 Scale (1.5 Kattai)    247
D3 Scale (2 Kattai)       236
D#3 Scale (2.5 Kattai)    189
C3 Scale (1 Kattai)       183
E3 Scale (3 Kattai)       124
F3 Scale (4 Kattai)        91
G#3 Scale (5.5 Kattai)     90
G3 Scale (5 Kattai)        89
A#3 Scale (6.5 Kattai)     81
F#3 Scale (4.5 Kattai)     62
B2 Scale (7 Kattai)        59
A3 Scale (6 Kattai)        57
A2 Scale (6 Kattai)        32
E4 Scale (3 Kattai)        21
D#4 Scale (2.5 Kattai)     15
C4 Scale (1 Kattai)        15
B3 Scale (7 Kattai)        12
C#4 Scale (1.5 Kattai)     12
D4 Scale (2 Kattai)        10
A#2 Scale (6.5 Kattai)      9
Name: 0, dtype: int64

In [ ]:
noList = ("A#2 Scale (6.5 Kattai)", "D4 Scale (2 Kattai)", "C#4 Scale (1.5 Kattai)", "B3 Scale (7 Kattai)", "C4 Scale (1 Kattai)",
          "D#4 Scale (2.5 Kattai)", "E4 Scale (3 Kattai)", "A2 Scale (6 Kattai)")

df = pd.DataFrame()

for i in range(1634):
  if not label[i] in noList:
    df = pd.concat((df, pd.DataFrame(dataset.iloc[i:i+1, :])))

In [ ]:
df.reset_index()
tonics = df[df. columns[0]]
tonics.value_counts()

C#3 Scale (1.5 Kattai)    247
D3 Scale (2 Kattai)       236
D#3 Scale (2.5 Kattai)    189
C3 Scale (1 Kattai)       183
E3 Scale (3 Kattai)       124
F3 Scale (4 Kattai)        91
G#3 Scale (5.5 Kattai)     90
G3 Scale (5 Kattai)        89
A#3 Scale (6.5 Kattai)     81
F#3 Scale (4.5 Kattai)     62
B2 Scale (7 Kattai)        59
A3 Scale (6 Kattai)        57
Name: 0, dtype: int64

In [ ]:
df = df.drop(labels = df.columns[0],axis = 1)

In [ ]:
df.reset_index(drop=True)

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.21420707,8.739072,5.4581776,0.12624136,8.633966,6.005421,5.3671145,5.8196044,6.815417,0.77815306,...,8.939017,6.01373,2.9354196,7.2605414,3.3460839,3.347998,4.0343165,4.451378,3.6681566,7.069213
1,0.19139859,8.72793,5.4421716,0.12790571,8.654089,6.005576,5.373312,5.804322,6.8137565,0.77869517,...,8.940513,6.016377,2.9174383,7.2737613,3.3376122,3.3539596,4.028694,4.47315,3.655445,7.066298
2,0.32489538,7.553173,5.5465374,0.5361066,7.054894,6.1986256,3.6050868,6.1132793,7.198174,1.187211,...,8.795121,5.866919,2.8955386,7.3232875,3.2719634,3.6217587,3.913939,4.4956546,3.502529,6.9241495
3,0.19774042,8.530767,5.3491335,0.15404506,8.283065,6.0297594,5.066106,5.872609,6.899556,0.8475405,...,8.936176,6.0069604,2.920703,7.2569437,3.3297422,3.3697774,4.0059447,4.467611,3.642208,7.0544457
4,0.47903278,6.9050937,5.873948,0.2699418,6.6134295,6.2527695,3.6795855,5.589656,7.1518235,1.1339802,...,8.910105,5.874103,2.930425,7.1691713,3.3336062,3.6457756,3.929723,4.4826846,3.5337646,6.9784174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,-0.2968913,10.372708,2.0733008,0.56054115,6.861363,6.04923,3.9645612,6.478813,7.183828,1.2605832,...,8.773013,6.1139903,2.750575,7.4332695,3.1950436,3.2866178,3.79234,4.759364,3.548786,6.866463
1504,0.25177035,4.8479657,2.2010782,0.5309143,7.0188174,7.5815096,6.312972,3.786211,7.0952587,2.4141276,...,8.492237,6.0313244,2.8835645,7.3376055,3.325283,3.410332,3.7955968,4.508328,3.552064,6.8394456
1505,-0.10077635,9.493347,3.1905453,0.35587102,7.3204455,6.149867,4.1113224,5.9045587,7.6174374,1.283629,...,9.008068,6.0275226,2.8792205,7.262005,3.3050323,3.3004897,3.9208515,4.467095,3.5787642,6.955029
1506,0.22782302,6.525382,1.1101161,0.8922466,7.057933,7.709207,5.940352,4.3674583,6.8488264,1.8107198,...,8.415417,6.088776,2.7777362,7.52872,3.1923866,3.3603911,3.669385,4.7074404,3.5860624,6.869464


# Split dataset

In [ ]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(df, tonics, test_size=0.3, random_state=109) # 70% training and 30% test
# random_state = a number means that this line of code will always produce the same split of data, instead of being random each time it is run
# see https://stackoverflow.com/questions/28064634/random-state-pseudo-random-number-in-scikit-learn for more

# Balance dataset

In [ ]:
# balance the dataset by undersampling the larger classes (tonics)
# see this tutorial for more info: https://towardsdatascience.com/how-to-balance-a-dataset-in-python-36dff9d12704
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
under_sampler = RandomUnderSampler(random_state=42)
X_res, y_res = under_sampler.fit_resample(X_train, y_train)
print(f"Training target statistics: {Counter(y_res)}")
print(f"Testing target statistics: {Counter(y_test)}")

Training target statistics: Counter({'A#3 Scale (6.5 Kattai)': 41, 'A3 Scale (6 Kattai)': 41, 'B2 Scale (7 Kattai)': 41, 'C#3 Scale (1.5 Kattai)': 41, 'C3 Scale (1 Kattai)': 41, 'D#3 Scale (2.5 Kattai)': 41, 'D3 Scale (2 Kattai)': 41, 'E3 Scale (3 Kattai)': 41, 'F#3 Scale (4.5 Kattai)': 41, 'F3 Scale (4 Kattai)': 41, 'G#3 Scale (5.5 Kattai)': 41, 'G3 Scale (5 Kattai)': 41})
Testing target statistics: Counter({'D3 Scale (2 Kattai)': 77, 'C#3 Scale (1.5 Kattai)': 71, 'D#3 Scale (2.5 Kattai)': 56, 'C3 Scale (1 Kattai)': 54, 'E3 Scale (3 Kattai)': 34, 'F3 Scale (4 Kattai)': 30, 'G3 Scale (5 Kattai)': 30, 'A#3 Scale (6.5 Kattai)': 25, 'G#3 Scale (5.5 Kattai)': 24, 'F#3 Scale (4.5 Kattai)': 20, 'B2 Scale (7 Kattai)': 18, 'A3 Scale (6 Kattai)': 14})


# **SVM - support vector machine**

Unbalanced

GridSearchCV: https://www.geeksforgeeks.org/svm-hyperparameter-tuning-using-gridsearchcv-ml/

In [ ]:
# Import svm model
from sklearn import svm

from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 0.5, 1.0, 1.5, 10, 100, 1000],
              'gamma': ['scale', 'auto', 1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['poly', 'sigmoid', 'linear', 'rbf'],
              'decision_function_shape': ['ovr']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 100, 'decision_function_shape': 'ovr', 'gamma': 'scale', 'kernel': 'poly'}
SVC(C=100, kernel='poly')


In [ ]:
from sklearn import metrics

grid_predictions = grid.predict(X_test)

# print classification report and accuracy, Matthews Correlaction Coefficient
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      1.00      1.00        14
   B2 Scale (7 Kattai)       1.00      0.67      0.80        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      1.00      1.00        77
   E3 Scale (3 Kattai)       0.92      1.00      0.96        34
F#3 Scale (4.5 Kattai)       0.95      0.95      0.95        20
   F3 Scale (4 Kattai)       1.00      0.97      0.98        30
G#3 Scale (5.5 Kattai)       1.00      1.00      1.00        24
   G3 Scale (5 Kattai)       0.91      1.00      0.95        30

              accuracy                           0.98       453
             macro avg       0.98      0.97      0.97       453
          weighted avg       0.98     

Balanced

In [ ]:
# Import svm model
from sklearn import svm

from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {'C': [0.1, 0.5, 1.0, 1.5, 10, 100, 1000],
              'gamma': ['scale', 'auto', 1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['poly', 'sigmoid', 'linear', 'rbf'],
              'decision_function_shape': ['ovr']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_res, y_res)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'C': 0.1, 'decision_function_shape': 'ovr', 'gamma': 1, 'kernel': 'poly'}
SVC(C=0.1, gamma=1, kernel='poly')


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      1.00      1.00        14
   B2 Scale (7 Kattai)       0.92      0.67      0.77        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      0.99      0.99        77
   E3 Scale (3 Kattai)       0.92      1.00      0.96        34
F#3 Scale (4.5 Kattai)       0.95      0.95      0.95        20
   F3 Scale (4 Kattai)       0.94      0.97      0.95        30
G#3 Scale (5.5 Kattai)       1.00      0.92      0.96        24
   G3 Scale (5 Kattai)       0.91      1.00      0.95        30

              accuracy                           0.98       453
             macro avg       0.97      0.96      0.96       453
          weighted avg       0.98     

# **KNN - k-nearest neighbors**

In [ ]:
# Scale the features using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Unbalanced

GridSearchCV: https://machinelearningknowledge.ai/knn-classifier-in-sklearn-using-gridsearchcv-with-example/#vii_Model_fitting_with_K-cross_Validation_and_GridSearchCV

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

# defining parameter range
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=10, scoring='accuracy', return_train_score=False, verbose=1)

# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             scoring='accuracy', verbose=1)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'n_neighbors': 1}
KNeighborsClassifier(n_neighbors=1)


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      1.00      1.00        14
   B2 Scale (7 Kattai)       1.00      0.67      0.80        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      1.00      1.00        77
   E3 Scale (3 Kattai)       0.87      1.00      0.93        34
F#3 Scale (4.5 Kattai)       0.95      0.95      0.95        20
   F3 Scale (4 Kattai)       0.97      0.97      0.97        30
G#3 Scale (5.5 Kattai)       1.00      1.00      1.00        24
   G3 Scale (5 Kattai)       1.00      1.00      1.00        30

              accuracy                           0.98       453
             macro avg       0.98      0.97      0.97       453
          weighted avg       0.98     

Balanced

In [ ]:
# Scale the features using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_res = scaler.fit_transform(X_res)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)

# defining parameter range
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=10, scoring='accuracy', return_train_score=False, verbose=1)

# fitting the model for grid search
grid.fit(X_res, y_res)

Fitting 10 folds for each of 30 candidates, totalling 300 fits


GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30]},
             scoring='accuracy', verbose=1)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'n_neighbors': 1}
KNeighborsClassifier(n_neighbors=1)


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       0.39      0.88      0.54        25
   A3 Scale (6 Kattai)       0.00      0.00      0.00        14
   B2 Scale (7 Kattai)       0.89      0.44      0.59        18
C#3 Scale (1.5 Kattai)       0.76      0.99      0.86        71
   C3 Scale (1 Kattai)       1.00      0.54      0.70        54
D#3 Scale (2.5 Kattai)       0.73      0.96      0.83        56
   D3 Scale (2 Kattai)       0.00      0.00      0.00        77
   E3 Scale (3 Kattai)       1.00      0.06      0.11        34
F#3 Scale (4.5 Kattai)       0.79      0.75      0.77        20
   F3 Scale (4 Kattai)       1.00      0.73      0.85        30
G#3 Scale (5.5 Kattai)       0.16      0.83      0.27        24
   G3 Scale (5 Kattai)       0.92      0.80      0.86        30

              accuracy                           0.59       453
             macro avg       0.64      0.58      0.53       453
          weighted avg       0.63     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# **DecisionTreeClassifier**

documentation: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

Unbalanced

GridSeachCV: https://plainenglish.io/blog/hyperparameter-tuning-of-decision-tree-classifier-using-gridsearchcv-2a6ebcaffeda

In [ ]:
# Import model
from sklearn import tree

# defining parameter range
param_grid = { 'criterion': ['gini', 'entropy'],
               'max_depth': range(1,10),
               'min_samples_split': range(1,10),
               'min_samples_leaf': range(1,5) }

grid = GridSearchCV(tree.DecisionTreeClassifier(random_state=3), param_grid, refit = True, cv=10, verbose=2)

# fitting the model for grid search
grid.fit(X_train, y_train)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'criterion': 'entropy', 'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 2}
DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=3,
                       random_state=3)


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       0.87      0.93      0.90        14
   B2 Scale (7 Kattai)       1.00      0.61      0.76        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       0.98      1.00      0.99        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       0.99      1.00      0.99        77
   E3 Scale (3 Kattai)       0.94      0.97      0.96        34
F#3 Scale (4.5 Kattai)       0.95      0.90      0.92        20
   F3 Scale (4 Kattai)       0.91      0.97      0.94        30
G#3 Scale (5.5 Kattai)       0.81      0.88      0.84        24
   G3 Scale (5 Kattai)       1.00      0.97      0.98        30

              accuracy                           0.96       453
             macro avg       0.95      0.93      0.94       453
          weighted avg       0.97     

Balanced

GridSeachCV: https://plainenglish.io/blog/hyperparameter-tuning-of-decision-tree-classifier-using-gridsearchcv-2a6ebcaffeda

In [ ]:
# Import model
from sklearn import tree
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = { 'criterion': ['gini', 'entropy'],
               'max_depth': range(1,10),
               'min_samples_split': range(1,10),
               'min_samples_leaf': range(1,5) }

grid = GridSearchCV(tree.DecisionTreeClassifier(random_state=3), param_grid, refit = True, cv=10, verbose=2)

# fitting the model for grid search
grid.fit(X_res, y_res)

Streaming output truncated to the last 5000 lines.
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=5; total time=   0.1s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_sampl

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
720 fits failed out of a total of 6480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _val

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=3),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10),
                         'min_samples_leaf': range(1, 5),
                         'min_samples_split': range(1, 10)},
             verbose=2)

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 3}
DecisionTreeClassifier(criterion='entropy', max_depth=7, min_samples_split=3,
                       random_state=3)


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       0.93      1.00      0.96        25
   A3 Scale (6 Kattai)       0.93      0.93      0.93        14
   B2 Scale (7 Kattai)       0.94      0.83      0.88        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      0.98      0.99        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      0.99      0.99        77
   E3 Scale (3 Kattai)       0.94      1.00      0.97        34
F#3 Scale (4.5 Kattai)       0.90      0.90      0.90        20
   F3 Scale (4 Kattai)       0.85      0.93      0.89        30
G#3 Scale (5.5 Kattai)       1.00      0.79      0.88        24
   G3 Scale (5 Kattai)       0.97      1.00      0.98        30

              accuracy                           0.97       453
             macro avg       0.95      0.95      0.95       453
          weighted avg       0.97     

# **RandomForestClassifier**

documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

Unbalanced

GridSeachCV: https://www.kaggle.com/code/sociopath00/random-forest-using-gridsearchcv/notebook

In [ ]:
# Import model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# defining parameter range
param_grid = {
    'n_estimators': [200, 500],
    'max_features': [None, 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid, cv=10)
grid.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [200, 500]})

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'criterion': 'entropy', 'max_depth': 6, 'max_features': 'log2', 'n_estimators': 500}
RandomForestClassifier(criterion='entropy', max_depth=6, max_features='log2',
                       n_estimators=500, random_state=42)


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       0.96      1.00      0.98        25
   A3 Scale (6 Kattai)       1.00      1.00      1.00        14
   B2 Scale (7 Kattai)       1.00      0.83      0.91        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      1.00      1.00        77
   E3 Scale (3 Kattai)       0.92      0.97      0.94        34
F#3 Scale (4.5 Kattai)       0.95      0.95      0.95        20
   F3 Scale (4 Kattai)       1.00      0.97      0.98        30
G#3 Scale (5.5 Kattai)       1.00      1.00      1.00        24
   G3 Scale (5 Kattai)       1.00      1.00      1.00        30

              accuracy                           0.99       453
             macro avg       0.98      0.98      0.98       453
          weighted avg       0.99     

Balanced

In [ ]:
# Import model
from sklearn.ensemble import RandomForestClassifier

# defining parameter range
param_grid = {
    'n_estimators': [64, 100, 200, 500],
    'max_features': [None, 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid=param_grid, cv=10)
grid.fit(X_res, y_res)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8],
                         'max_features': [None, 'sqrt', 'log2'],
                         'n_estimators': [64, 100, 200, 500]})

In [ ]:
# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'criterion': 'entropy', 'max_depth': 6, 'max_features': None, 'n_estimators': 100}
RandomForestClassifier(criterion='entropy', max_depth=6, max_features=None,
                       random_state=42)


In [ ]:
grid_predictions = grid.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, grid_predictions))
print("Accuracy:", metrics.accuracy_score(y_test, grid_predictions))
print("MCC:", metrics.matthews_corrcoef(y_test, grid_predictions))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       0.96      1.00      0.98        25
   A3 Scale (6 Kattai)       1.00      0.93      0.96        14
   B2 Scale (7 Kattai)       1.00      0.83      0.91        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      0.99      0.99        77
   E3 Scale (3 Kattai)       1.00      1.00      1.00        34
F#3 Scale (4.5 Kattai)       0.95      0.95      0.95        20
   F3 Scale (4 Kattai)       0.83      0.97      0.89        30
G#3 Scale (5.5 Kattai)       1.00      0.92      0.96        24
   G3 Scale (5 Kattai)       1.00      1.00      1.00        30

              accuracy                           0.98       453
             macro avg       0.98      0.97      0.97       453
          weighted avg       0.98     

# **GradientBoostingClassifier**

documentation: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

Unbalanced

In [ ]:
# defining parameter range
param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Create Gradient Boosting classifer object
clf = GradientBoostingClassifier(random_state=42, loss='log_loss', learning_rate=0.2,
                                 n_estimators=400, subsample=1.0, criterion='friedman_mse',
                                 min_samples_split=100, min_samples_leaf=0.1,
                                 max_depth=8, max_features='sqrt')

# Train classifer
clf = clf.fit(X_train,y_train)

In [ ]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
# print classification report
print(metrics.classification_report(y_test, y_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("MCC:", metrics.matthews_corrcoef(y_test, y_pred))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      1.00      1.00        14
   B2 Scale (7 Kattai)       1.00      0.72      0.84        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       0.97      1.00      0.99        77
   E3 Scale (3 Kattai)       0.94      1.00      0.97        34
F#3 Scale (4.5 Kattai)       0.95      0.95      0.95        20
   F3 Scale (4 Kattai)       0.97      0.97      0.97        30
G#3 Scale (5.5 Kattai)       1.00      1.00      1.00        24
   G3 Scale (5 Kattai)       1.00      1.00      1.00        30

              accuracy                           0.98       453
             macro avg       0.98      0.97      0.98       453
          weighted avg       0.98     

Balanced

In [ ]:
# Create Gradient Boosting classifer object
clf = GradientBoostingClassifier(random_state=42, loss='log_loss', learning_rate=0.2,
                                 n_estimators=200, subsample=0.8, criterion='friedman_mse',
                                 min_samples_split=2, min_samples_leaf=0.2,
                                 max_depth=5, max_features='sqrt')

# Train classifer
clf = clf.fit(X_res,y_res)

In [ ]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, y_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("MCC:", metrics.matthews_corrcoef(y_test, y_pred))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      1.00      1.00        14
   B2 Scale (7 Kattai)       1.00      0.83      0.91        18
C#3 Scale (1.5 Kattai)       1.00      1.00      1.00        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      0.99      0.99        77
   E3 Scale (3 Kattai)       1.00      1.00      1.00        34
F#3 Scale (4.5 Kattai)       0.90      0.95      0.93        20
   F3 Scale (4 Kattai)       0.85      0.97      0.91        30
G#3 Scale (5.5 Kattai)       1.00      0.92      0.96        24
   G3 Scale (5 Kattai)       1.00      1.00      1.00        30

              accuracy                           0.98       453
             macro avg       0.98      0.97      0.97       453
          weighted avg       0.98     

# **AdaBoost**

Unbalanced

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=500, learning_rate=0.2, algorithm='SAMME.R',
                         random_state=3)

clf.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=0.2, n_estimators=500, random_state=3)

In [ ]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, y_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("MCC:", metrics.matthews_corrcoef(y_test, y_pred))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      0.79      0.88        14
   B2 Scale (7 Kattai)       0.43      0.83      0.57        18
C#3 Scale (1.5 Kattai)       1.00      0.72      0.84        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       0.98      1.00      0.99        56
   D3 Scale (2 Kattai)       1.00      1.00      1.00        77
   E3 Scale (3 Kattai)       0.93      0.74      0.82        34
F#3 Scale (4.5 Kattai)       0.79      0.95      0.86        20
   F3 Scale (4 Kattai)       0.45      0.47      0.46        30
G#3 Scale (5.5 Kattai)       0.89      0.71      0.79        24
   G3 Scale (5 Kattai)       0.64      0.90      0.75        30

              accuracy                           0.86       453
             macro avg       0.84      0.84      0.83       453
          weighted avg       0.89     

Balanced

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=150, learning_rate=0.2, algorithm='SAMME.R',
                         random_state=3)

clf.fit(X_res, y_res)

AdaBoostClassifier(learning_rate=0.2, n_estimators=150, random_state=3)

In [ ]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)

# print classification report
print(metrics.classification_report(y_test, y_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("MCC:", metrics.matthews_corrcoef(y_test, y_pred))

                        precision    recall  f1-score   support

A#3 Scale (6.5 Kattai)       1.00      1.00      1.00        25
   A3 Scale (6 Kattai)       1.00      0.93      0.96        14
   B2 Scale (7 Kattai)       0.24      0.89      0.38        18
C#3 Scale (1.5 Kattai)       1.00      0.30      0.46        71
   C3 Scale (1 Kattai)       1.00      1.00      1.00        54
D#3 Scale (2.5 Kattai)       1.00      0.86      0.92        56
   D3 Scale (2 Kattai)       1.00      0.97      0.99        77
   E3 Scale (3 Kattai)       0.94      0.97      0.96        34
F#3 Scale (4.5 Kattai)       1.00      0.95      0.97        20
   F3 Scale (4 Kattai)       0.65      1.00      0.79        30
G#3 Scale (5.5 Kattai)       1.00      0.92      0.96        24
   G3 Scale (5 Kattai)       1.00      0.93      0.97        30

              accuracy                           0.85       453
             macro avg       0.90      0.89      0.86       453
          weighted avg       0.94     